In [1]:
import os
import warnings

warnings.simplefilter(action="ignore")
os.environ["GRPC_VERBOSITY"] = "NONE"

# Multi Agent Building

Dependency injection shown in [02.tool_calling_and_dependency_injection.ipynb](./02.tool_calling_and_dependency_injection.ipynb) allow us to build multi agent with intuitive coding. `Agent` class has the argument `subagents` that accepts `list[Agent]` typed variable, and subagents are dynamically converted to tools which invokes agent using dependency injection feature. This notebook shows examples of multi-agent building. 

# Prerequisites

Please make sure your environmental variables and dependencies are ready to use LLM services. Name of the environmental variables is arbitraray because langrila modules accepts that name as an argument.

In [2]:
from dotenv import load_dotenv

load_dotenv("../../.env_api")

True

# Import modules

In [3]:
from langrila import Agent, InMemoryConversationMemory
from langrila.anthropic import AnthropicClient
from langrila.google import GoogleClient
from langrila.openai import OpenAIClient

# Instantiate client

In [4]:
# For OpenAI
openai_client = OpenAIClient(api_key_env_name="OPENAI_API_KEY")

# For Azure OpenAI
azure_openai_client = OpenAIClient(
    api_key_env_name="AZURE_API_KEY",
    api_type="azure",
    azure_api_version="2024-11-01-preview",
    azure_endpoint_env_name="AZURE_ENDPOINT",
    azure_deployment_id_env_name="AZURE_DEPLOYMENT_ID",
)

# For Gemini on Google AI Studio
google_dev_client = GoogleClient(api_key_env_name="GEMINI_API_KEY")

# For Gemini on Google Cloud VertexAI
vertexai_client = GoogleClient(
    api_type="vertexai",
    project_id_env_name="GOOGLE_CLOUD_PROJECT",
    location="us-central1",
)

# For Claude of Anthropic
anthropic_client = AnthropicClient(api_key_env_name="ANTHROPIC_API_KEY")

# For Claude of Amazon Bedrock
claude_bedrock_client = AnthropicClient(
    api_type="bedrock",
    aws_access_key_env_name="AWS_ACCESS_KEY",
    aws_secret_key_env_name="AWS_SECRET_KEY",
    aws_region_env_name="AWS_REGION",
)

# Define tools

Using dummy tools.

In [5]:
import random
from typing import Literal


def power_disco_ball(power: bool) -> bool:
    """
    Powers the spinning dissco ball.

    Parameters
    ----------
    power : bool
        Whether to power the disco ball or not.

    Returns
    ----------
    bool
        Whether the disco ball is spinning or not.
    """
    return f"Disco ball is {'spinning!' if power else 'stopped.'}"


def start_music() -> str:
    """
    Turn on the music. The genre, BPM, and volume are randomly selected.

    Returns
    ----------
    str
        A message indicating that the music is starting.
    """
    music_genre = random.choice(["rock", "pop", "jazz", "classical", "hip-hop"])
    volume = random.uniform(0.2, 1.0)
    bpm = 120
    return f"Starting music! Genre: {music_genre}, BPM: {bpm}, Volume: {volume}"


def change_bpm(new_bpm: int) -> int:
    """
    Change the BPM of the music.

    Parameters
    ----------
    new_bpm : int
        The new BPM to set the music.

    Returns
    ----------
    int
        The new BPM of the music.
    """
    return f"Changing BPM to {new_bpm}"


def change_music(
    genre: Literal["rock", "pop", "jazz", "classical", "hip-hop"],
    bpm: int,
) -> str:
    """
    Change the music genre and BPM.

    Parameters
    ----------
    genre : str
        The genre of music to play. Should be one of "rock", "pop", "jazz", "classical", or "hip-hop".
    bpm : int
        The BPM of the music.

    Returns
    ----------
    str
        A message indicating that the music has been changed.
    """
    return f"Changing music to {genre} with BPM {bpm}"


def turn_light_on() -> str:
    """
    Turn the lights on.

    Returns
    ----------
    str
        A message indicating that the lights are turning on.
    """
    brightness = random.uniform(0.5, 1.0)
    return "Lights are now on! Brightness: {:.2f}".format(brightness)


def dim_lights(brightness: float) -> bool:
    """
    Dim the lights.

    Parameters
    ----------
    brightness : float
        The brightness level to set the lights. Should be between 0 and 1.

    Returns
    ----------
    bool
        Whether the lights were successfully dimmed.
    """
    return f"Lights are now set to {brightness}"


def adjust_volume(volume: float) -> bool:
    """
    Adjust the volume of the music.

    Parameters
    ----------
    volume : float
        The volume level to set the music. Should be between 0 and 1.

    Returns
    ----------
    bool
        Whether the volume was successfully adjusted.
    """
    return f"Volume is now set to {volume}"


def stop_music() -> str:
    """
    Stop the music.

    Returns
    ----------
    str
        A message indicating that the music is stopping.
    """
    return "Stopping music!"


def stop_disco_ball() -> str:
    """
    Stop the disco ball power and spinning.

    Returns
    ----------
    str
        A message indicating that the disco ball is stopping.
    """
    return "Stopping disco ball! Powered off and stopped spinning."


def adjust_lights(brightness: float) -> bool:
    """
    Adjust the brightness of the lights.

    Parameters
    ----------
    brightness : float
        The brightness level to set the lights. Should be between 0 and 1.

    Returns
    ----------
    bool
        Whether the lights were successfully brightened.
    """
    return f"Lights are now set to {brightness}"

# Multi-Agent

`Agent` class accepts `subagents` argument which generates dynamically tools to run agent. Langrila supports multi agent with multi client. In langrila, we can build orchestrator-typed multi-agent, not graph-based multi-agent. The orchestrator routes the execution of tools to task-specific agents, aggregates the results, and outputs the final answer.

## Task specific agent. 

In [6]:
lights_agent = Agent(
    client=vertexai_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=[turn_light_on, adjust_lights],
)

disco_ball_agent = Agent(
    client=anthropic_client,
    model="claude-3-5-sonnet-20240620",
    temperature=0.0,
    tools=[power_disco_ball, stop_disco_ball],
    max_tokens=500,
)

music_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=[start_music, change_music, adjust_volume],
)

## Orchestrator agent.

The thing you have to do for building multi-agent is to pass agents to the orchestrator agent. Planning mode is supported for multi agent.

In [8]:
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    planning=True,
)

In [9]:
prompt = "Turn this place into a party mood!"

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 20:10:44] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood!\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - change_music: Change the music genre and BPM.\n  - adjust_volume: Adjust the volume of the music.\n')]
[2025-01-02 20:10:44] INFO | root: Generating text
[2025-01-02 20:10:48] DEBUG | Response: [TextResponse(text='### Plan 

As you can see in the log message above, the orchestrator agent assigned subdivided task to each subagents and finally generated answer.

In [10]:
print(response.contents[0].text)

The party mood has been successfully set! Here's what has been done:

1. **Lights**: The lights are on and ready.
2. **Brightness Adjustment**: Please provide a brightness level between 0 and 1 to adjust the lights to a vibrant level.
3. **Disco Ball**: The disco ball is spinning, adding a fun visual element to the atmosphere.
4. **Music**: Upbeat music is playing at a BPM of 120 with a volume level of approximately 0.53.

Let me know the brightness level you'd like to set or if there's anything else you'd like to do!


Usage is collected all over the agent including subagents.

In [11]:
list(response.usage.items())

[('music_agent',
  Usage(model_name='gpt-4o-mini-2024-07-18', prompt_tokens=441, output_tokens=43)),
 ('lights_agent',
  Usage(model_name='gemini-2.0-flash-exp', prompt_tokens=61, output_tokens=23)),
 ('disco_ball_agent',
  Usage(model_name='claude-3-5-sonnet-20240620', prompt_tokens=990, output_tokens=149)),
 ('root',
  Usage(model_name='gpt-4o-mini-2024-07-18', prompt_tokens=1958, output_tokens=424))]

Top-level orchestrator name is `root`.

# State in the agent

Agent state in existing agent frameworks has some issues on readability, special argument, and traceability. Ideally, state should be expressed by both the dependencies between agents and response schema, no special manner should be taken. It means the state is updated by llm based on the conversation history and response schema while its scope is limited by the agent's dependencies.

In langrila, the combination of the dependencies between the agents and structured output can take the place of the state in agent. Note that planning mode is supported even if multi agent case.

## Response schema

In [12]:
from enum import Enum

from pydantic import BaseModel, Field


class DiscoBallSchema(BaseModel):
    power: bool = Field(..., description="Whether to power the disco ball.")
    spinning: bool = Field(..., description="Whether the disco ball is spinning.")


class MusicGenre(str, Enum):
    rock = "rock"
    pop = "pop"
    jazz = "jazz"
    classical = "classical"
    hip_hop = "hip-hop"


class MusicSchema(BaseModel):
    genre: MusicGenre = Field(
        ...,
        description="The genre of music to play.",
    )
    bpm: int = Field(
        ...,
        description="The BPM of the music.",
        ge=60,
        le=180,
    )
    volume: float = Field(
        ...,
        description="The volume level to set the music to.",
        ge=0,
        le=1,
    )


class LightsSchema(BaseModel):
    brightness: float = Field(
        ...,
        description="The brightness level to set the lights to.",
        ge=0,
        le=1,
    )


class ResponseSchema(BaseModel):
    disco_ball: DiscoBallSchema = Field(..., description="The disco ball settings.")
    music: MusicSchema = Field(..., description="The music settings.")
    lights: LightsSchema = Field(..., description="The lights settings.")

## Task specific agent.

In [18]:
lights_agent = Agent(
    client=google_dev_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=[turn_light_on, adjust_lights],
    response_schema_as_tool=LightsSchema,  # The state of the lights.
)

disco_ball_agent = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    tools=[power_disco_ball, stop_disco_ball],
    max_tokens=500,
    response_schema_as_tool=DiscoBallSchema,  # The state of the disco ball.
)


music_power_agent = Agent(
    client=openai_client,
    model="gpt-4o-mini-2024-07-18",
    temperature=0.0,
    tools=[start_music],
)

music_control_agent = Agent(
    client=google_dev_client,
    model="gemini-2.0-flash-exp",
    temperature=0.0,
    tools=[change_music, adjust_volume, change_bpm],
    planning=True,  # Planning mode is enable for the subagent.
)

## Orchestrator agent

In [19]:
# Orchestrator agent as a subagent
music_agent = Agent(
    client=anthropic_client,
    model="claude-3-5-sonnet-20240620",
    temperature=0.0,
    subagents=[music_power_agent, music_control_agent],
    max_tokens=500,
    response_schema_as_tool=MusicSchema,  # The state of the music.
)

# Orchestrator agent
orchestrator = Agent(
    client=openai_client,
    model="gpt-4o-2024-11-20",
    temperature=0.0,
    subagents=[lights_agent, disco_ball_agent, music_agent],
    conversation_memory=InMemoryConversationMemory(),
    response_schema_as_tool=ResponseSchema,  # The state of the party.
    planning=True,
)

In [20]:
prompt = "Turn this place into a party mood."

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 20:21:49] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')]
[2025-01-02 20:21:

In [21]:
list(response.usage.items())

[('music_power_agent',
  Usage(model_name='gpt-4o-mini-2024-07-18', prompt_tokens=175, output_tokens=39)),
 ('music_agent',
  Usage(model_name='claude-3-5-sonnet-20240620', prompt_tokens=3347, output_tokens=424)),
 ('lights_agent',
  Usage(model_name='gemini-2.0-flash-exp', prompt_tokens=466, output_tokens=14)),
 ('disco_ball_agent',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=801, output_tokens=53)),
 ('root',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=2599, output_tokens=354))]

In [22]:
print(response.contents[0].text)

{"disco_ball": {"power": true, "spinning": true}, "music": {"genre": "pop", "bpm": 120, "volume": 0.5}, "lights": {"brightness": 0.8}}


Validation response schema

In [23]:
valid_resposne = ResponseSchema.model_validate_json(response.contents[0].text)
valid_resposne.model_dump()

{'disco_ball': {'power': True, 'spinning': True},
 'music': {'genre': <MusicGenre.pop: 'pop'>, 'bpm': 120, 'volume': 0.5},
 'lights': {'brightness': 0.8}}

Conversation memory stores the conversation history in the top-level orchestrator agent.

In [24]:
orchestrator.load_history()

[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')], name=None)

Next turn prompt

In [25]:
prompt = "Prefer to a jazz music with a calm tempo. Please change the music to jazz and stop the disco ball."

response = orchestrator.generate_text(prompt=prompt)

[2025-01-02 20:25:54] DEBUG | Prompt: [TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nPrefer to a jazz music with a calm tempo. Please change the music to jazz and stop the disco ball.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - 

In [26]:
list(response.usage.items())

[('music_control_agent',
  Usage(model_name='gemini-2.0-flash-exp', prompt_tokens=3452, output_tokens=111)),
 ('disco_ball_agent',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=1958, output_tokens=33)),
 ('music_agent',
  Usage(model_name='claude-3-5-sonnet-20240620', prompt_tokens=4082, output_tokens=229)),
 ('root',
  Usage(model_name='gpt-4o-2024-11-20', prompt_tokens=4599, output_tokens=200))]

Validation response schema

In [27]:
valid_resposne = ResponseSchema.model_validate_json(response.contents[0].text)
valid_resposne.model_dump()

{'disco_ball': {'power': False, 'spinning': False},
 'music': {'genre': <MusicGenre.jazz: 'jazz'>, 'bpm': 80, 'volume': 0.5},
 'lights': {'brightness': 0.8}}

In [28]:
orchestrator.load_history()

[Prompt(type='Prompt', role='user', contents=[TextPrompt(text='Please make a concise plan to answer the following question/requirement, considering the conversation history.\nYou can invoke the sub-agents or tools to answer the questions/requirements shown in the capabilities section.\nAgent has no description while the tools have a description.\n\nQuestion/Requirement:\nTurn this place into a party mood.\n\nCapabilities:\n- lights_agent\n  - turn_light_on: Turn the lights on.\n  - adjust_lights: Adjust the brightness of the lights.\n- disco_ball_agent\n  - power_disco_ball: Powers the spinning dissco ball.\n  - stop_disco_ball: Stop the disco ball power and spinning.\n- music_agent\n  - music_power_agent\n    - start_music: Turn on the music. The genre, BPM, and volume are randomly selected.\n  - music_control_agent\n    - change_music: Change the music genre and BPM.\n    - adjust_volume: Adjust the volume of the music.\n    - change_bpm: Change the BPM of the music.\n')], name=None)